<a href="https://colab.research.google.com/github/gunavc/RecommenderEngine/blob/master/RecommenderEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -O moviedataset.zip https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
print('unziping ...')

!unzip -o -j moviedataset.zip

--2022-06-09 09:27:10--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%205/data/moviedataset.zip
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 160301210 (153M) [application/zip]
Saving to: ‘moviedataset.zip’

moviedataset.zip    100%[===================>] 152.88M  9.31MB/s    in 7.9s    

2022-06-09 09:27:19 (19.3 MB/s) - ‘moviedataset.zip’ saved [160301210/160301210]

unziping ...
Archive:  moviedataset.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: README.txt              
  inflating: tags.csv        

In [ ]:
#Dataframe manipulation library
import pandas as pd
from math import sqrt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#Storing the movie information into a pandas dataframe
movies_df = pd.read_csv('movies.csv')
#Storing the user information into a pandas dataframe
ratings_df = pd.read_csv('ratings.csv')
movies_df.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: FutureWarning: The default value of regex will change from True to False in a future version.
  import sys


,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


In [ ]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


In [ ]:
#Copying the movie dataframe into a new one since we won't need to use the genre information in our first case.
moviesWithGenres_df = movies_df.copy()

#For every row in the dataframe, iterate through the list of genres and place a 1 into the corresponding column
for index, row in movies_df.iterrows():
    for genre in row['genres']:
        moviesWithGenres_df.at[index, 
                                     genre] = int(1)
       
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
moviesWithGenres_df = moviesWithGenres_df.fillna(0)
moviesWithGenres_df

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34203,151697,Grand Slam,[Thriller],1967,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34204,151701,Bloodmoney,[(no genres listed)],2010,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
34205,151703,The Butterfly Circus,[Drama],2009,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34206,151709,Zero,"[Drama, Sci-Fi]",2015,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,169,2.5,1204927694
1,1,2471,3.0,1204927438
2,1,48516,5.0,1204927435
3,2,2571,3.5,1436165433
4,2,109487,4.0,1436165496


In [ ]:
#Drop removes a specified row or column from a dataframe
ratings_df = ratings_df.drop('timestamp', 1)
ratings_df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,userId,movieId,rating
0,1,169,2.5
1,1,2471,3.0
2,1,48516,5.0
3,2,2571,3.5
4,2,109487,4.0


In [ ]:
userInput = [
            {'title':'Breakfast Club, The', 'rating':5},
            {'title':'Toy Story', 'rating':3.5},
            {'title':'Jumanji', 'rating':2},
            {'title':"Pulp Fiction", 'rating':5},
            {'title':'Akira', 'rating':4.5}
         ] 
inputMovies = pd.DataFrame(userInput)
print(inputMovies.shape)

(5, 2)


In [ ]:
#Filtering out the movies by title

inputId = movies_df[movies_df['title'].isin(inputMovies['title'].tolist())]
#Then merging it so we can get the movieId. It's implicitly merging it by title.
inputMovies = pd.merge(inputId, inputMovies)
#Dropping information we won't use from the input dataframe
inputMovies = inputMovies.drop('genres', 1).drop('year', 1)
#Final input dataframe
inputMovies
print(inputId)

      movieId                title  \
0           1            Toy Story   
1           2              Jumanji   
293       296         Pulp Fiction   
1246     1274                Akira   
1885     1968  Breakfast Club, The   

                                                 genres  year  
0     [Adventure, Animation, Children, Comedy, Fantasy]  1995  
1                        [Adventure, Children, Fantasy]  1995  
293                    [Comedy, Crime, Drama, Thriller]  1994  
1246             [Action, Adventure, Animation, Sci-Fi]  1988  
1885                                    [Comedy, Drama]  1985  


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


In [ ]:
#Filtering out the movies from the input
userMovies = moviesWithGenres_df[moviesWithGenres_df['movieId'].isin(inputMovies['movieId'].tolist())]
userMovies

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,...,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
293,296,Pulp Fiction,"[Comedy, Crime, Drama, Thriller]",1994,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1246,1274,Akira,"[Action, Adventure, Animation, Sci-Fi]",1988,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1885,1968,"Breakfast Club, The","[Comedy, Drama]",1985,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
#Dropping unwanted columns
genres_df = moviesWithGenres_df.drop(["title","genres","year","movieId"], axis=1)
genres_df.reset_index()
print(genres_df.head())

input_df = userMovies.drop(["title","genres","year","movieId"], axis=1)
input_df.head()
print(input_df.shape)

   Adventure  Animation  Children  Comedy  Fantasy  Romance  Drama  Action  \
0        1.0        1.0       1.0     1.0      1.0      0.0    0.0     0.0   
1        1.0        0.0       1.0     0.0      1.0      0.0    0.0     0.0   
2        0.0        0.0       0.0     1.0      0.0      1.0    0.0     0.0   
3        0.0        0.0       0.0     1.0      0.0      1.0    1.0     0.0   
4        0.0        0.0       0.0     1.0      0.0      0.0    0.0     0.0   

   Crime  Thriller  Horror  Mystery  Sci-Fi  IMAX  Documentary  War  Musical  \
0    0.0       0.0     0.0      0.0     0.0   0.0          0.0  0.0      0.0   
1    0.0       0.0     0.0      0.0     0.0   0.0          0.0  0.0      0.0   
2    0.0       0.0     0.0      0.0     0.0   0.0          0.0  0.0      0.0   
3    0.0       0.0     0.0      0.0     0.0   0.0          0.0  0.0      0.0   
4    0.0       0.0     0.0      0.0     0.0   0.0          0.0  0.0      0.0   

   Western  Film-Noir  (no genres listed)  
0     

In [ ]:
#Weight Genre Matrix
wg_matrix = pd.DataFrame(userInput).drop("title", axis=1)
wg_matrix

print(wg_matrix.shape)
wg_matrix = wg_matrix.T
print(wg_matrix.shape)


(5, 1)
(1, 5)


In [ ]:
#User Profile
np.array(wg_matrix)
np.array(input_df)

user_profile = np.dot(wg_matrix, input_df)
print(user_profile)
print(user_profile.shape)

[[13.5 10.   8.5 11.5  8.5  0.   6.5  5.   2.   2.   0.   0.   5.   0.
   0.   0.   0.   0.   0.   0. ]]
(1, 20)


In [ ]:
#normalisation
sum = np.sum(user_profile)
print(sum)
normalised_array = np.divide(user_profile, sum)
print("normalised array: ",normalised_array)

72.5
normalised array:  [[0.1862069  0.13793103 0.11724138 0.15862069 0.11724138 0.
  0.08965517 0.06896552 0.02758621 0.02758621 0.         0.
  0.06896552 0.         0.         0.         0.         0.
  0.         0.        ]]


In [ ]:
genres_df["Adventure"]

0        1.0
1        1.0
2        0.0
3        0.0
4        0.0
        ... 
34203    0.0
34204    0.0
34205    0.0
34206    0.0
34207    0.0
Name: Adventure, Length: 34208, dtype: float64

In [ ]:
#Recommendation matrix
recommendation_matrix = pd.DataFrame(genres_df["Adventure"]*normalised_array[0,0])
column_name = genres_df.columns

for i in range(1,20):
  temp = (genres_df[column_name[i]]*normalised_array[0,i])
  recommendation_matrix = recommendation_matrix.join(temp)

display(recommendation_matrix)




,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed)
0,0.186207,0.137931,0.117241,0.158621,0.117241,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.186207,0.000000,0.117241,0.000000,0.117241,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.158621,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.158621,0.000000,0.0,0.089655,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.158621,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34203,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.027586,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34204,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34205,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.089655,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0
34206,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.089655,0.0,0.0,0.000000,0.0,0.0,0.068966,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
recommendation_matrix["sum"] = recommendation_matrix[column_name].sum(axis=1)
recommendation_matrix

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,...,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed),sum
0,0.186207,0.137931,0.117241,0.158621,0.117241,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.717241
1,0.186207,0.000000,0.117241,0.000000,0.117241,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.420690
2,0.000000,0.000000,0.000000,0.158621,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.158621
3,0.000000,0.000000,0.000000,0.158621,0.000000,0.0,0.089655,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.248276
4,0.000000,0.000000,0.000000,0.158621,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.158621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34203,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.027586,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027586
34204,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
34205,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.089655,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.089655
34206,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.089655,0.0,0.0,0.000000,...,0.0,0.068966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.158621


In [ ]:
#Inserting the movieId and title columns into the recommendation matrix
recommendation_matrix.insert(0,"movieId",moviesWithGenres_df["movieId"])
recommendation_matrix.insert(1,"title",moviesWithGenres_df["title"])

recommendation_matrix

,movieId,title,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,...,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed),sum
0,1,Toy Story,0.186207,0.137931,0.117241,0.158621,0.117241,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.717241
1,2,Jumanji,0.186207,0.000000,0.117241,0.000000,0.117241,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.420690
2,3,Grumpier Old Men,0.000000,0.000000,0.000000,0.158621,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.158621
3,4,Waiting to Exhale,0.000000,0.000000,0.000000,0.158621,0.000000,0.0,0.089655,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.248276
4,5,Father of the Bride Part II,0.000000,0.000000,0.000000,0.158621,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.158621
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34203,151697,Grand Slam,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.027586
34204,151701,Bloodmoney,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
34205,151703,The Butterfly Circus,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.089655,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.089655
34206,151709,Zero,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.089655,0.0,...,0.0,0.068966,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.158621


In [ ]:
#Sorting the recommendation matrix by sum in ascending order
new_mat = recommendation_matrix.sort_values(by=['sum'],ascending=False)
new_mat = new_mat.reset_index()
new_mat

,index,movieId,title,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,...,Mystery,Sci-Fi,IMAX,Documentary,War,Musical,Western,Film-Noir,(no genres listed),sum
0,8605,26093,"Wonderful World of the Brothers Grimm, The",0.186207,0.137931,0.117241,0.158621,0.117241,0.0,0.089655,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.806897
1,8783,26340,"Twelve Tasks of Asterix, The (Les douze travau...",0.186207,0.137931,0.117241,0.158621,0.117241,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.786207
2,22881,108932,The Lego Movie,0.186207,0.137931,0.117241,0.158621,0.117241,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.786207
3,11751,51939,TMNT (Teenage Mutant Ninja Turtles),0.186207,0.137931,0.117241,0.158621,0.117241,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.786207
4,11716,51632,Atlantis: Milo's Return,0.186207,0.137931,0.117241,0.158621,0.117241,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.786207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34203,27580,127861,The Last Gladiators,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
34204,14406,72032,Forever,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
34205,27576,127845,"Back to the Garden, Flower Power Comes Full Ci...",0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
34206,1294,1323,Amityville 3-D,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [ ]:
#Printing the recommended movies
print("You might also like \n")
for i in range(20):
  print(new_mat["title"][i])

You might also like 

Wonderful World of the Brothers Grimm, The
Twelve Tasks of Asterix, The (Les douze travaux d'Astérix)
The Lego Movie
TMNT (Teenage Mutant Ninja Turtles)
Atlantis: Milo's Return
Home
Space Jam
Robots
Revolutionary Girl Utena: Adolescence of Utena (a.k.a. Revolutionary Girl Utena the Movie) (Shoujo kakumei Utena: Adolescence mokushiroku)
Who Framed Roger Rabbit?
Chicken Little
Aqua Teen Hunger Force Colon Movie Film for Theaters
Free Birds
Meet the Robinsons
Wizards of Waverly Place: The Movie
Futurama: Bender's Game
Mars Needs Moms
Underdog
Super Mario Bros.
Shrek Forever After (a.k.a. Shrek: The Final Chapter)
